## Gradient Boosting

In [5]:
# model_gb.py - VERSIÓN COMPLETA PARA NOTEBOOK
from pathlib import Path
import joblib
from datetime import datetime
import os
import pandas as pd

# Data management - VERSIÓN NOTEBOOK
CURRENT_DIR = Path.cwd()
MODELS_DIR = CURRENT_DIR / "trained_models"
MODELS_DIR.mkdir(exist_ok=True, parents=True)

class GradientBoostingPurchaseModel:
    def __init__(self, dataset_version="v1.0", **params):
        self.model_type = "GradientBoosting"
        self.params = params
        self.dataset_version = dataset_version
        self.model = self._build_model()
        self.training_date = datetime.now()
        self.feature_importance_ = None
        self.is_fitted = False
        
    def _build_model(self):
        try:
            from xgboost import XGBClassifier
            default_params = {
                'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 6,
                'subsample': 0.8, 'random_state': 42, 'eval_metric': 'logloss'
            }
            final_params = {**default_params, **self.params}
            return XGBClassifier(**final_params)
        except ImportError:
            from sklearn.ensemble import GradientBoostingClassifier
            print("INFO: Usando GradientBoosting de sklearn")
            default_params = {
                'n_estimators': 100, 'learning_rate': 0.1, 
                'max_depth': 6, 'random_state': 42
            }
            final_params = {**default_params, **self.params}
            return GradientBoostingClassifier(**final_params)

    def fit(self, X, y, X_val=None, y_val=None):
        if X_val is not None and y_val is not None and hasattr(self.model, 'fit'):
            try:
                self.model.fit(X, y, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)
            except TypeError:
                print("Usando entrenamiento sin early stopping")
                self.model.fit(X, y)
        else:
            self.model.fit(X, y)
        
        if hasattr(self.model, 'feature_importances_'):
            self.feature_importance_ = self.model.feature_importances_
        
        self.is_fitted = True
        return self

    def predict(self, X):
        if not self.is_fitted:
            raise ValueError("Modelo no entrenado. Llama a fit() primero.")
        return self.model.predict(X)

    def predict_proba(self, X):
        if not self.is_fitted:
            raise ValueError("Modelo no entrenado. Llama a fit() primero.")
        return self.model.predict_proba(X)

    def get_config(self):
        config = {
            'model_type': self.model_type, 'dataset_version': self.dataset_version,
            'hyperparameters': self.params, 'is_fitted': self.is_fitted,
            'training_date': self.training_date.strftime("%Y-%m-%d %H:%M:%S")
        }
        if self.is_fitted:
            config['n_features'] = getattr(self.model, 'n_features_in_', None)
        return config

    def get_feature_importance(self, feature_names=None):
        if self.feature_importance_ is None:
            return None
        if feature_names is None:
            feature_names = [f'feature_{i}' for i in range(len(self.feature_importance_))]
        importance_df = pd.DataFrame({
            'feature': feature_names, 'importance': self.feature_importance_
        }).sort_values('importance', ascending=False)
        return importance_df

    def save(self, prefix: str):
        now = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"GB_{prefix}_{now}.pkl"
        filepath = MODELS_DIR / filename
        filepath.parent.mkdir(parents=True, exist_ok=True)
        joblib.dump(self, filepath)
        print(f"GradientBoosting model saved to {filepath}")
        return filepath

    def load(self, filename: str):
        filepath = MODELS_DIR / filename
        model = joblib.load(filepath)
        print(f"GradientBoosting model loaded from {filepath}")
        return model

    def __repr__(self):
        return f"GradientBoostingPurchaseModel(fitted={self.is_fitted})"

# PARA PROBAR EN NOTEBOOK
print("✅ GradientBoosting model_gb.py cargado correctamente!")
model = GradientBoostingPurchaseModel(n_estimators=100, learning_rate=0.1, max_depth=6)
print(f"Modelo creado: {model}")
print(f"Config: {model.get_config()}")

✅ GradientBoosting model_gb.py cargado correctamente!
INFO: Usando GradientBoosting de sklearn
Modelo creado: GradientBoostingPurchaseModel(fitted=False)
Config: {'model_type': 'GradientBoosting', 'dataset_version': 'v1.0', 'hyperparameters': {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 6}, 'is_fitted': False, 'training_date': '2025-10-08 11:32:36'}


## Random Forest